## !!Before Running This Notebook!!

go to the /data/ section of this repo and review the README, as well as the data_creation notebook located there

In [1]:
import pandas as pd
import gzip
import json
import re
import os
import pickle
import nltk
import re
import multiprocessing

from nltk.corpus import stopwords

nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package words to
[nltk_data]     /Users/kyledecember1/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kyledecember1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from tqdm import tqdm
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics import accuracy_score, f1_score

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
cores = multiprocessing.cpu_count()

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

def get_recs(model, user_input, n=5):
    description = user_input.split(' ')
    desc_vec = model.infer_vector(description)
    recs = model.docvecs.most_similar([desc_vec])[:n]
    return recs

def show_game_desc(recs, df):
    games = {}
    for rec in recs:
        games[rec[0]] = df[df['product_id'] == rec[0]]['title']
    return games

## Import  Data

In [4]:
#assign data filepaths to variables
games_path = os.path.join(os.pardir, os.pardir, 'data/games.csv')
reviews_path = os.path.join(os.pardir, os.pardir, 'data/subsample_agg_reviews.p')

In [5]:
#import df_games 
df_games = pd.read_csv(games_path)

#drop null values
df_games['id'].dropna(inplace=True)

# create product_id column in df_games, based on str-formatted id column
df_games['product_id'] = df_games['id'].astype(int).astype(str)

In [6]:
agg_df = pd.DataFrame()

with open(reviews_path, 'rb') as fp:
    loaded_file = pickle.load(fp)
    
agg_df['product_id'] = list(loaded_file.keys())
agg_df['reviews'] = list(loaded_file.values())

## Modeling & Evaluation

    Please note the subjective nature of the evaluation process; without a focus group it was impossible to obtain information on whether or not the recommendations were relevant to the text provided as the user's "Ideal Video Game Description"
    
    Evaluation was performed by reviewing the descriptions of the games provided, and determining if they matched key words in the descriptions. Ultimately, each model was given a Relevance Score, which is simply the percentage of recommendations that were determined to be relevant.